In [4]:
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
face_detection_model = "models/face_detection_yunet_2022mar.onnx"
top_k = 5000
nms_threshold = 0.3
score_threshold = 0.9
face_recognition_model = "models/face_recognition_sface_2021dec.onnx"

detector = cv2.FaceDetectorYN.create(face_detection_model,
                                    "",
                                    (320, 320),
                                    score_threshold,
                                    nms_threshold,
                                    top_k)
recognizer = cv2.FaceRecognizerSF.create(
        face_recognition_model,"")



path = "dataset"

# loop over persons in the dataset
for folder in os.listdir(path):
    print("Open embedding file: {}_embeddings.xml".format(folder))

    # open embedding file
    f = cv2.FileStorage("{}_embeddings.xml".format(folder), cv2.FileStorage_WRITE)
    i = 0
    # loop over images and calculate embeddings
    for img in tqdm(os.listdir(os.path.join(path, folder))):
        image = cv2.imread(os.path.join(path, folder, img))
        detector.setInputSize((image.shape[1], image.shape[0]))
        try:
            faces = detector.detect(image)
            face_align = recognizer.alignCrop(image, faces[1][0])
            face_feature = recognizer.feature(face_align)
            f.write("_{}".format(i), face_feature) #
            i+=1
        except:
            continue
    f.release()
print("Done!")

Open embedding file: yechiel_embeddings.xml


100%|██████████| 25/25 [00:21<00:00,  1.18it/s]


Open embedding file: avraham_embeddings.xml


100%|██████████| 28/28 [00:36<00:00,  1.31s/it]

Done!


In [50]:
# save embedings
s = cv2.FileStorage("embs.xml", cv2.FileStorage_WRITE)
for i in range(5):
    s.write("_{}".format(i), face1_feature)
s.release()

In [ ]:
# read embedings
s = cv2.FileStorage("embs.xml", cv2.FileStorage_READ)
t = cv2.Mat()
for i in range(5):
    t.push_back(s.getNode(str(i)).mat())  
s.release()